# **Settings**

In [3]:
import os
import warnings
warnings.filterwarnings('ignore')                       # warning 출력 false

import numpy as np 
import pandas as pd
import re

# **1. Model Settings**

In [4]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments

# model setting
model_path = './kcbert2'

# arguments
args = ClassificationDeployArguments(
    pretrained_model_name = 'beomi/kcbert-base',
    downstream_model_dir = model_path,
    max_seq_length = 128
)


downstream_model_checkpoint_fpath: ./kcbert2\epoch=5-val_loss=0.01.ckpt


# **2. Tokenizer**

In [5]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case = False
)

# **3. Load Checkpoint**

In [6]:
# Load checkpoint
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device('cpu')
)

# **4. Model Config**

In [7]:
from transformers import BertConfig, BertForSequenceClassification

# 모델 생성 및 초기화
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel()
)

model = BertForSequenceClassification(pretrained_model_config)

# **5. Model**

In [8]:
# 모델 불러오기
model.load_state_dict({k.replace('model.',''): v for k,v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

# 6. Inference

## **1) 채팅글 감정 정보**

In [59]:
def chat_preprocessing(file_path):
    # 채팅 불러오기
    with open(file_path,'r', encoding='utf-8') as f:
        chat_data = f.read().strip().split('\n')
    # print(chat_data)

    # 채팅 분리
    speak_list = []
    chat_list = []
    for i, sent in enumerate(chat_data):
        sent = sent.strip()
        p = re.compile('\[([^]]+)\]')
        speak = p.findall(sent)
        if speak == []:
            speak = speak_list[-1]
        else:
            speak = speak[0]

        chat = re.sub('\[([^]]+)\]','',sent).strip()
        chat = re.sub('[^가-힣\s]','',chat).strip()

        if i > 0 and speak == speak_list[-1]:
            chat_list[-1] = chat_list[-1]+ ' ' + chat
        else:
            chat_list.append(chat)
            speak_list.append(speak)

    # 내 것만
    my_list = []
    for y, m in zip(speak_list, chat_list):
        if y == '나' and len(m) > 1:
            my_list.append(m)

    return {'total': {'users':speak_list,'chat':chat_list}, 'user':my_list}

In [60]:
result = chat_preprocessing('./kakaotalk/전남친.txt')
for user, chat in zip(result['total']['users'],result['total']['chat']):
    print(f'{user}: {chat}')

상대방: 야 나 차단했냐 전화 좀 받아봐 나 진짜 할 말있어서 그래
나: 니 목소리 듣기도 싫으니까 꺼져
상대방: 진짜 중요한 일이라고 빨리 받아봐 이게 마지막이야
나: 
상대방: 혹시 이번주에 시간돼
나: 왜
상대방: 이런말하기 좀그렇지만 예전에 내가 사준 아이패드 돌려주라 나 그거 할부 아직 안끝낫거든
나: 선물로 준 거 아니었어
상대방: 그건 그런데 나는 아직도 할부갚고 있는데 아이패드는 너가 쓰고잇자나 이번주에 시간되면 아이패드좀 돌려주라
나: 택배로 보낼게 주소 알려줘
상대방: 응 아 그리고 혹시나해서 말하는데 착불은 치사한거 알지
나: 하 너 정말 구질구질하다 다신 연락하지마


In [61]:
my_list = result['user']
# 연인싸움
for text in my_list:
    print(text)
    print(f'>>> {inference_fn(text)}')

니 목소리 듣기도 싫으니까 꺼져
>>> 분노
선물로 준 거 아니었어
>>> 당황
택배로 보낼게 주소 알려줘
>>> 상처
하 너 정말 구질구질하다 다신 연락하지마
>>> 슬픔


## **2) 노래가사 감정 정보**

In [10]:
train_data = pd.read_csv('./data2/train_data.csv')
val_data = pd.read_csv('./data2/val_data.csv')
print(train_data.shape, val_data.shape)

(453965, 4) (56248, 4)


In [63]:
train_data['emotion'].unique()

for emotion in train_data['emotion'].unique():
    sub_emotion = train_data[train_data['emotion']==emotion]['sensitivity'].unique()
    print(f'{emotion}: {sub_emotion}')

Happy: ['고맙다' '굉장하다' '만족스럽다' '뭉클하다' '반갑다' '부유하다' '사랑스럽다' '영광스럽다' '자랑스럽다' '자유롭다'
 '즐겁다' '행복하다' '홀가분하다' '화목하다' '흐뭇하다']
Sad: ['가엾다' '괴롭다' '그립다' '보잘것없다' '불쌍하다' '불행하다' '뼈아프다' '서럽다' '시무룩하다' '쓸쓸하다' '아쉽다'
 '안타깝다' '억울하다']
Angry: ['경멸스럽다' '기막히다' '끔찍하다' '무례하다' '밉다' '배은망덕하다' '부당하다' '뻔뻔스럽다' '아니꼽다' '악랄하다'
 '억울하다' '언짢다' '짜증스럽다' '치욕스럽다' '한심하다']
Anxious: ['갑갑하다' '고통스럽다' '꺼림칙하다' '다급하다' '두렵다' '불확실하다' '수상하다' '심란하다' '아득하다' '약하다'
 '외롭다' '위험하다' '의심스럽다' '조마조마하다' '혼란하다']
Hurt: ['고통스럽다' '괴롭다' '냉정하다' '매정하다' '못되다' '무관심하다' '밉다' '불행하다' '뼈아프다' '서럽다' '섭섭하다'
 '슬프다' '쓰리다']
Embarrassed: ['갑작스럽다' '곤란하다' '급하다' '기막히다' '난데없다' '남사스럽다' '망하다' '버겁다' '부끄럽다' '아찔하다'
 '어리둥절하다' '엉뚱하다' '창피하다' '혼란하다']
Neutrality: ['긴밀하다' '녹녹하다' '담백하다' '대동소이하다' '둥그스름하다' '뚜렷하다' '얕다' '어렴풋하다' '엄밀하다' '엄중하다'
 '예사롭다' '자세하다']


In [64]:
import pickle
with open('./data2/encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)
encoder.classes_

array(['Angry', 'Anxious', 'Embarrassed', 'Happy', 'Hurt', 'Neutrality',
       'Sad'], dtype=object)

## **2) 노래가사 감정 추론**

In [39]:
# 추론
def inference_fn(sentence):
    emotion_list = ['분노', '불안', '당황', '기쁨', '상처', '중립','슬픔']
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding = 'max_length',
        truncation=True
    )

    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k,v in inputs.items()})
        prob = outputs.logits.softmax(dim=1)
        positive_prob = round(prob[0][1].item(),4)
        negative_prob = round(prob[0][0].item(),4)
        pred = torch.argmax(prob).item()
        
        return emotion_list[pred]
    
# 추론
def lyrics_preprocessing(lyrics):
    lyrics = re.sub('[^ㄱ-ㅣ가-힣]',' ',lyrics).split('  ')
    lyrics = [x .strip() for x in lyrics if x != '']
    lyrics = ' '.join(list(set(lyrics)))
    
    return lyrics

def inference_fn_lyrics(lyrics):
    emotion_list = ['분노', '불안', '당황', '기쁨', '상처', '중립','슬픔']
    
    sentence = lyrics_preprocessing(lyrics)
    if len(sentence) == 0:
        return 'X'
    
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding = 'max_length',
        truncation=True
    )

    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k,v in inputs.items()})
        prob = outputs.logits.softmax(dim=1)
        pred = torch.argmax(prob).item()
        
        return emotion_list[pred]

In [82]:
songs = pd.read_csv('./lyricsDB.csv')
songs

,title-singer,lyrics,new_lyrics
0,그대라는 시-태연 (TAEYEON),언제부터인지 그대를 보면\n운명이라고 느꼈던 걸까\n밤하늘의 별이 빛난 것처럼\n오...,언제부터인지 그대를 보면 운명이라고 느꼈던 걸까 밤하늘의 별이 빛난 것처럼 오랫동안...
1,면역력-하성운,오늘따라 유난히 목소리 잠겨\n내 맘 밤새 건조했던 탓일까\n매일같이 신경 써 조심...,오늘따라 유난히 목소리 잠겨 내 맘 밤새 건조했던 탓일까 매일같이 신경 써 조심을 ...
2,"다시 난, 여기-백예린 (Yerin Baek)",나의 마음이 움직여\n그냥 길을 따라 걸어가도\n다시 여기 너의 앞에\n두려웠던 나...,나의 마음이 움직여 그냥 길을 따라 걸어가도 다시 여기 너의 앞에 두려웠던 나의 맘...
3,기대했단 말야-JOO,기다렸어 난 잠이 들때마다\n연락이 오길 기도했어\n힘들었지만 참을 수 있었어\n애...,기다렸어 난 잠이 들때마다 연락이 오길 기도했어 힘들었지만 참을 수 있었어 애써 웃...
4,그 끝에 그대-청하,다가서면 멀어지는 저 꿈결처럼\n그댄 어디로 가나요\n내 맘속에 살고 있는 그대란 ...,다가서면 멀어지는 저 꿈결처럼 그댄 어디로 가나요 내 맘속에 살고 있는 그대란 사람...
...,...,...,...
6753,Up!-Kep1er (케플러),Listen\nI do I do 오늘도 내일도\nDon’t waste it\nNa ...,오늘도 내일도 주문을 외우고 더 멋진 제일 멋진 색깔로 울려 퍼지는 출발 직전 아찔...
6754,WA DA DA-Kep1er (케플러),Hey\nIt’s you\n& I\nLet’s start\nN N Now 도착한 이...,도착한 이 나의 신호를 따라 꿈꿔온 모험 가까이 있어 있어 한 발 한 발 내디뎌 갈...
6755,Giddy-Kep1er (케플러),Give me give me some\nGive me give me some\nGi...,우주를 뒤흔든 날 끌어당기는 강력한 너란 중력에 땅이 울리고 심장이 떨리는 널 향한...
6756,Take It-IVE (아이브),Just take it take it\nYeah take it take it\nJu...,대체 뭘 망설이니 답답한 너드처럼 밀고 당길 뻔한 루틴까지 재미없어 난 필요 없는 ...


In [83]:
emotion_list = []
cnt = 0
print('Start!', end=' ')
for lyrics in songs['new_lyrics']:
    emotion = inference_fn_lyrics(lyrics)
    emotion_list.append(emotion)
    
    cnt += 1
    if cnt % 200 == 0:
        print(f'> {cnt}',end=' ')
print('End!')

Start! > 200 > 400 > 600 > 800 > 1000 > 1200 > 1400 > 1600 > 1800 > 2000 > 2200 > 2400 > 2600 > 2800 > 3000 > 3200 > 3400 > 3600 > 3800 > 4000 > 4200 > 4400 > 4600 > 4800 > 5000 > 5200 > 5400 > 5600 > 5800 > 6000 > 6200 > 6400 > 6600 End!


In [86]:
# 데이터 저장
songs['emotion'] = emotion_list
songs = songs[songs['emotion']!='X']
songs.to_csv('./lyricsDB_emotion.csv',index=False)

In [87]:
song_emotion = {}
for title, emotion in zip(songs['title-singer'],songs['emotion']):
#     print(title, emotion)
#     break
    song_emotion[title] = emotion

import pickle
with open('./song2emotion.pkl', 'wb') as f:
    pickle.dump(song_emotion, f)

In [75]:
# 랜덤으로 확인하기
n = 100
rand_idx = np.random.randint(0,songs.shape[0],n)

for idx in rand_idx:
    title = songs.iloc[idx,0]
    lyrics = songs.iloc[idx,1]
    emotion = inference_fn_lyrics(lyrics)
    print(f'[{title}]의 감정은 {emotion}입니다.')

[연애-Sondia]의 감정은 분노입니다.
[Singer Songwriter (feat. 김이지)-크루셜스타 (CRUCiAL STAR)]의 감정은 기쁨입니다.
[일년이면-휘성 (Realslow)]의 감정은 슬픔입니다.
[위로-권진아]의 감정은 상처입니다.
[Baby Baby-포맨 (4MEN)]의 감정은 슬픔입니다.
[흐린 가을 하늘에 편지를 써-동물원]의 감정은 슬픔입니다.
[Wayne Remix (feat. G2, Huckleberry P & P-TYPE)-JUSTHIS & Paloalto (저스디스 & 팔로알토)]의 감정은 슬픔입니다.
[Red Light-f(x)]의 감정은 분노입니다.
[오늘... 그녀가-루그]의 감정은 슬픔입니다.
[좋아해서 미안-Stray Kids (스트레이 키즈)]의 감정은 슬픔입니다.
[고백-장범준]의 감정은 슬픔입니다.
[난 알아요-서태지와 아이들]의 감정은 슬픔입니다.
[Run Devil Run-소녀시대 (GIRLS' GENERATION)]의 감정은 분노입니다.
[Moonlight-Sondia]의 감정은 슬픔입니다.
[그 해 여름-인피니트]의 감정은 상처입니다.
[내 삶의 반-한경일]의 감정은 슬픔입니다.
[귀를 기울이면 (LOVE WHISPER)-여자친구 (GFRIEND)]의 감정은 기쁨입니다.
[It Will Be-김성규]의 감정은 슬픔입니다.
[끼부리지마-WINNER]의 감정은 분노입니다.
[잊혀지길-범키]의 감정은 상처입니다.
[바다보러갈래 (SEE SEA)-효린]의 감정은 기쁨입니다.
[비가와 (Rain)-소유 (SOYOU), 백현 (BAEKHYUN)]의 감정은 상처입니다.
[이 밤을 살아가는 너에게 (Vocal. 지유)-타임브릿지]의 감정은 슬픔입니다.
[What is Love?-TWICE (트와이스)]의 감정은 상처입니다.
[오늘은 가지마-벤]의 감정은 슬픔입니다.
[고백-이수훈]의 감정은 상처입니다.
[예뻤어-DAY6 (데이식스)]의 감정은 상처입니다.
[사나이 가는 길 (폼생폼사)-젝스키스]의 감정은 슬픔입니다.